In [ ]:
import torch
import torch.onnx
import torchvision
from torchsummary import summary

In [ ]:
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.randn(1, 3, 224, 224)
dummy_input_2 = torch.randn(1, 600, 600, 3)
dummy_input_3 = torch.randn(1, 224, 224, 3)

In [ ]:
# Obtain your model, it can be also constructed in your script explicitly
# model = torchvision.models.alexnet(pretrained=True)
model = torchvision.models.resnet18(pretrained=True)

In [ ]:
class ModelWrapper(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = torchvision.models.resnet18(pretrained=True)   

    def forward(self, x): 
        # x = torch.permute(x, (0, 3, 1, 2))
        x = self.backbone(x)
        return x

In [ ]:
model = ModelWrapper()

In [ ]:
# model

In [ ]:
dummy_output = model(dummy_input)

In [ ]:
dummy_output.shape

In [ ]:
#
# Computes the number of trainable parameters
#
# Sourse:
# https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325
#
def get_num_of_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
get_num_of_params(model)

In [ ]:
def get_size_of_model(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    
    print('model size: {:.3f}MB'.format(size_all_mb))

In [ ]:
get_size_of_model(model)

In [ ]:
summary(model, (3, 224, 224))

In [ ]:
# Invoke export
input_names = ["input_1"]
output_names = ["output_1"]

torch.onnx.export(
    model,
    dummy_input, 
    "models/resnet18chw.onnx",
    opset_version=12,
    input_names=input_names,
    output_names=output_names
)

# ONNX to ORT

In [ ]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort --help

In [ ]:
# MODEL_NAME="./models/resnet18.onnx"
MODEL_NAME='/Users/user006/Developer/models/yunet/yan_1class.onnx'

In [ ]:
!python3 -m onnxruntime.tools.convert_onnx_models_to_ort {MODEL_NAME}

In [ ]:
python -m onnxruntime.tools.convert_onnx_models_to_ort ./models/idp/resnet18_mobile_numbers_quantized/resnet18_mobile_numbers_quantized.onnx

# Inspecting model

In [ ]:
import onnx

In [ ]:
!ls models

In [ ]:
# Load the ONNX model
onnx_model = onnx.load("models/capturing_2305.onnx")

# Check that the IR is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
# Print a human readable representation of the graph
# print(onnx.helper.printable_graph(onnx_model.graph))

In [ ]:
!pwd

In [ ]:
!cd ../capturing/ && pwd

In [ ]:
!cd ../capturing/ && python inference.py

# pytorch to coreml
https://coremltools.readme.io/docs/pytorch-conversion

In [ ]:
# !pip install coremltools

In [ ]:
import coremltools as ct

In [ ]:
model = model.eval()

In [ ]:
traced_model = torch.jit.trace(model, dummy_input)
out = traced_model(dummy_input)

In [ ]:
out.shape

In [ ]:
# traced_model

In [ ]:
# Using image_input in the inputs parameter:
# Convert to Core ML using the Unified Conversion API.
coreml_model = ct.convert(
    traced_model,
#     inputs=[ct.TensorType(shape=dummy_input.shape)]
    inputs=[ct.ImageType(shape=dummy_input.shape)]
)

In [ ]:
# Save the converted model.
coreml_model.save("resnet18.mlmodel")